In [2]:
import sys, os, re
from typing import *
from collections import deque, defaultdict
from typing import Any, Iterator
from dataclasses import dataclass
sys.path = (["../src/", "../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path
import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import Subtype, Instrument, Signal
from qubx.core.interfaces import IStrategy, IStrategyContext, TriggerEvent
from qubx.core.series import Quote, Bar
from qubx.pandaz.utils import *
from qubx.core.account import AccountProcessor
from qubx.core.context import StrategyContext
from qubx.core.loggers import InMemoryLogsWriter, StrategyLogging

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.data.helpers import loader
from qubx.data.readers import DataReader, DataTransformer, AsPandasFrame, AsTimestampedRecords, RestoreTicksFromOHLC
from qubx.backtester.queue import SimulatedDataQueue, DataLoader
from qubx.backtester.simulated_data import IteratedDataStreamsSlicer, IterableSimulatorData

from qubx.backtester.simulator import simulate, SimulatedTrading, SimulatedExchange, find_instruments_and_exchanges

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [3]:
s1 = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
s2 = lookup.find_symbol("BINANCE.UM", "ETHUSDT")
s3 = lookup.find_symbol("BINANCE.UM", "LTCUSDT")

In [3]:
ld = loader("BINANCE.UM", "4h", source="csv::../tests/data/csv_1h", n_jobs=1)
# ld = loader("BINANCE.UM", "1h", source="mqdb::xlydian-data", n_jobs=1)

### Test DataLoader

In [1258]:
q = SimulatedDataQueue()
l1 = DataLoader(RestoreTicksFromOHLC(), ld, s1, warmup_period="1d", timeframe="1h")
l2 = DataLoader(RestoreTicksFromOHLC(), ld, s2, warmup_period="1d", timeframe="1h")
q += l1
q += l2

In [1259]:
k = 0
itr = q.create_iterable("2023-07-01", "2023-07-02")
for t in itr:
    print(f"{pd.Timestamp(t[2].time, 'ns')}     \t {t[0].symbol}\t{t[2]} / {t[1]}")
    if k == 10:
        print("+ " * 10)
        q += (l3:=DataLoader(RestoreTicksFromOHLC(), ld, s3, warmup_period="3d", timeframe="1h"))

    if k == 30:
        print("- " * 10)
        # q -= l3
    k+=1

2023-06-30 00:00:00.001000     	 BTCUSDT	[2023-06-30T00:00:00.001000000]	30439.90000 (1000000000.0) | 30439.90000 (1000000000.0) / hist_ohlc
2023-06-30 00:00:00.001000     	 ETHUSDT	[2023-06-30T00:00:00.001000000]	1851.18000 (1000000000.0) | 1851.18000 (1000000000.0) / hist_ohlc
2023-06-30 00:24:00     	 BTCUSDT	[2023-06-30T00:24:00.000000000]	30500.00000 (1000000000.0) | 30500.00000 (1000000000.0) / hist_ohlc
2023-06-30 00:24:00     	 ETHUSDT	[2023-06-30T00:24:00.000000000]	1856.26000 (1000000000.0) | 1856.26000 (1000000000.0) / hist_ohlc
2023-06-30 00:36:00     	 BTCUSDT	[2023-06-30T00:36:00.000000000]	30385.00000 (1000000000.0) | 30385.00000 (1000000000.0) / hist_ohlc
2023-06-30 00:36:00     	 ETHUSDT	[2023-06-30T00:36:00.000000000]	1846.64000 (1000000000.0) | 1846.64000 (1000000000.0) / hist_ohlc
2023-06-30 00:59:59.999000     	 BTCUSDT	[2023-06-30T00:59:59.999000000]	30431.30000 (1000000000.0) | 30431.30000 (1000000000.0) / hist_ohlc
2023-06-30 00:59:59.999000     	 ETHUSDT	[2023-

---

### <font color='gold'>Impl. multi-symbol</font>

In [104]:
@dataclass
class DummyTimeEvent:
    time: pd.Timestamp
    data: str

    @staticmethod
    def from_dict(data: dict[str | pd.Timedelta, str], start: str) -> list["DummyTimeEvent"]:
        _t0 = pd.Timestamp(start)
        return [DummyTimeEvent((_t0 + pd.Timedelta(t)).as_unit("ns").asm8.item(), d) for t, d in data.items()]

    @staticmethod
    def from_seq(start: str, n: int, ds: str, pfx: str) -> list["DummyTimeEvent"]:
        return DummyTimeEvent.from_dict({s * pd.Timedelta(ds): pfx for s in range(n+1)}, start)

    def __repr__(self) -> str:
        return f"{pd.Timestamp(self.time, 'ns')} -> ({self.data})"

#### - First impl -

In [146]:
slicer = IteratedDataStreamsSlicer()

data1 = [
    DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1"),
    DummyTimeEvent.from_seq("2020-01-01 00:11", 10, "1Min", "A2"),
]

data2 = [
    DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1"),
    DummyTimeEvent.from_seq("2020-01-01 00:16", 10, "1Min", "B2"),
]

data3 = [
    DummyTimeEvent.from_seq("2020-01-01 00:08", 10, "1Min", "C1"),
    DummyTimeEvent.from_seq("2020-01-01 00:19", 10, "1Min", "C2"),
]

slicer += {
    'it3': iter(data3),
    'it2': iter(data2),
    'it1': iter(data1),
}

r = []
for t in slicer:
    if not t: continue
    print(f"{pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)

2020-01-01 00:00:00 | id=it1 | A1
2020-01-01 00:01:00 | id=it1 | A1
2020-01-01 00:02:00 | id=it1 | A1
2020-01-01 00:03:00 | id=it1 | A1
2020-01-01 00:04:00 | id=it1 | A1
2020-01-01 00:05:00 | id=it1 | A1
2020-01-01 00:05:00 | id=it2 | B1
2020-01-01 00:06:00 | id=it1 | A1
2020-01-01 00:06:00 | id=it2 | B1
2020-01-01 00:07:00 | id=it1 | A1
2020-01-01 00:07:00 | id=it2 | B1
2020-01-01 00:08:00 | id=it1 | A1
2020-01-01 00:08:00 | id=it2 | B1
2020-01-01 00:08:00 | id=it3 | C1
2020-01-01 00:09:00 | id=it1 | A1
2020-01-01 00:09:00 | id=it2 | B1
2020-01-01 00:09:00 | id=it3 | C1
2020-01-01 00:10:00 | id=it1 | A1
2020-01-01 00:10:00 | id=it2 | B1
2020-01-01 00:10:00 | id=it3 | C1
2020-01-01 00:11:00 | id=it1 | A2
2020-01-01 00:11:00 | id=it2 | B1
2020-01-01 00:11:00 | id=it3 | C1
2020-01-01 00:12:00 | id=it1 | A2
2020-01-01 00:12:00 | id=it2 | B1
2020-01-01 00:12:00 | id=it3 | C1
2020-01-01 00:13:00 | id=it1 | A2
2020-01-01 00:13:00 | id=it2 | B1
2020-01-01 00:13:00 | id=it3 | C1
2020-01-01 00:

#### <font color='#00ff00'>- New impl test -</font>

In [145]:
slicer = IteratedDataStreamsSlicer()

slicer += {
    "set.A": iter([DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1")]),
    "set.C": iter([DummyTimeEvent.from_seq("2020-01-01 00:01", 10, "1Min", "C1")]),
    "set.D": iter([DummyTimeEvent.from_seq("2020-01-01 00:03", 10, "1Min", "D1")]),
    "set.E": iter([DummyTimeEvent.from_seq("2020-01-01 00:03", 10, "1Min", "E1")]),
    "set.B": iter([DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1")]),
}

r = []
for t in slicer:
    if not t:
        continue
    print(f"{pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)

2020-01-01 00:00:00 | id=set.A | A1
2020-01-01 00:01:00 | id=set.A | A1
2020-01-01 00:01:00 | id=set.C | C1
2020-01-01 00:02:00 | id=set.A | A1
2020-01-01 00:02:00 | id=set.C | C1
2020-01-01 00:03:00 | id=set.A | A1
2020-01-01 00:03:00 | id=set.C | C1
2020-01-01 00:03:00 | id=set.D | D1
2020-01-01 00:03:00 | id=set.E | E1
2020-01-01 00:04:00 | id=set.A | A1
2020-01-01 00:04:00 | id=set.C | C1
2020-01-01 00:04:00 | id=set.D | D1
2020-01-01 00:04:00 | id=set.E | E1
2020-01-01 00:05:00 | id=set.A | A1
2020-01-01 00:05:00 | id=set.C | C1
2020-01-01 00:05:00 | id=set.D | D1
2020-01-01 00:05:00 | id=set.E | E1
2020-01-01 00:05:00 | id=set.B | B1
2020-01-01 00:06:00 | id=set.A | A1
2020-01-01 00:06:00 | id=set.C | C1
2020-01-01 00:06:00 | id=set.D | D1
2020-01-01 00:06:00 | id=set.E | E1
2020-01-01 00:06:00 | id=set.B | B1
2020-01-01 00:07:00 | id=set.A | A1
2020-01-01 00:07:00 | id=set.C | C1
2020-01-01 00:07:00 | id=set.D | D1
2020-01-01 00:07:00 | id=set.E | E1
2020-01-01 00:07:00 | id=set

In [140]:
print(r)

['A1', 'A1', 'C1', 'A1', 'C1', 'A1', 'C1', 'D1', 'E1', 'A1', 'C1', 'D1', 'E1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'C1', 'D1', 'E1', 'B1', 'D1', 'E1', 'B1', 'D1', 'E1', 'B1', 'B1', 'B1']


#### Add / remove on the fly

In [147]:
slicer = IteratedDataStreamsSlicer()

data1 = [
    DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1"),
    DummyTimeEvent.from_seq("2020-01-01 00:11", 10, "1Min", "A2"),
]

data2 = [
    DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1"),
    DummyTimeEvent.from_seq("2020-01-01 00:16", 10, "1Min", "B2"),
]

data3 = [
    DummyTimeEvent.from_seq("2020-01-01 00:08", 10, "1Min", "C1"),
    DummyTimeEvent.from_seq("2020-01-01 00:19", 10, "1Min", "C2"),
]

data4 = [
    DummyTimeEvent.from_seq("2020-01-01 00:10", 10, "1Min", "D1"),
]

slicer += {
    '1': iter(data1),
    '2': iter(data2),
    '3': iter(data3),
}

r, k = [], 0
for t in slicer:
    if not t: continue
    print(f"{k:3d}: {pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)
    if k == 3: slicer.remove('1')
    if k == 11: slicer += {'10': iter(data4)}
    k += 1

  0: 2020-01-01 00:00:00 | id=1 | A1
  1: 2020-01-01 00:01:00 | id=1 | A1
  2: 2020-01-01 00:02:00 | id=1 | A1
  3: 2020-01-01 00:03:00 | id=1 | A1
  4: 2020-01-01 00:05:00 | id=2 | B1
  5: 2020-01-01 00:06:00 | id=2 | B1
  6: 2020-01-01 00:07:00 | id=2 | B1
  7: 2020-01-01 00:08:00 | id=2 | B1
  8: 2020-01-01 00:08:00 | id=3 | C1
  9: 2020-01-01 00:09:00 | id=2 | B1
 10: 2020-01-01 00:09:00 | id=3 | C1
 11: 2020-01-01 00:10:00 | id=2 | B1
 12: 2020-01-01 00:10:00 | id=3 | C1
 13: 2020-01-01 00:10:00 | id=10 | D1
 14: 2020-01-01 00:11:00 | id=3 | C1
 15: 2020-01-01 00:11:00 | id=10 | D1
 16: 2020-01-01 00:11:00 | id=2 | B1
 17: 2020-01-01 00:12:00 | id=3 | C1
 18: 2020-01-01 00:12:00 | id=10 | D1
 19: 2020-01-01 00:12:00 | id=2 | B1
 20: 2020-01-01 00:13:00 | id=3 | C1
 21: 2020-01-01 00:13:00 | id=10 | D1
 22: 2020-01-01 00:13:00 | id=2 | B1
 23: 2020-01-01 00:14:00 | id=3 | C1
 24: 2020-01-01 00:14:00 | id=10 | D1
 25: 2020-01-01 00:14:00 | id=2 | B1
 26: 2020-01-01 00:15:00 | id=3 |

In [13]:
print(r)

['A1', 'A1', 'A1', 'A1', 'B1', 'B1', 'B1', 'B1', 'C1', 'B1', 'C1', 'B1', 'C1', 'D1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B2', 'C1', 'D1', 'B2', 'C1', 'D1', 'B2', 'C2', 'D1', 'B2', 'C2', 'D1', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'C2', 'C2']


### <font color='#ff0050'>Subscriptions blotter</fint>

In [4]:
class TestA(IStrategy):
    timeframe: str = "1d"

    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(Subtype.OHLC[self.timeframe])

    def on_event(self, ctx: IStrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        if event.data and event.type == "event":
            signal = event.data.get("order")
            if signal is not None and event.instrument:
                return [event.instrument.signal(signal)]
        return None

In [5]:
ld = loader("BINANCE.UM", "1h", source="csv::../tests/data/csv_1h", n_jobs=1)

In [6]:
strat = TestA(timeframe="1h")
trading_service = SimulatedTrading("binance.um", "vip0_usdt")
broker = SimulatedExchange("binance.um", trading_service, ld)
account = AccountProcessor( account_id=trading_service.get_account_id(), base_currency='USDT', initial_capital=1000)

2024-12-03 17:16:08.060 [ 🐞 ] (helpers) TestA new parameters:
	set timeframe <- 1h


1970-01-01 00:00:00.000 [ℹ️] SimulatedData.binance.um initialized


In [7]:
ctx = StrategyContext(
    strategy=strat,
    broker=broker,
    account=account,
    instruments=[s1, s2],
    logging=StrategyLogging(InMemoryLogsWriter("test", "test", "0")),
)
ctx.start()

 >> get_subscribed_instruments __all__
 >>> get_subscriptions None
 >> get_subscribed_instruments ohlc(1h)
 >> get_subscribed_instruments None
 ~~~ Warming up {}
 >> get_subscribed_instruments ohlc(1h)
>>>> subscribe: ohlc(1h) -> {CRYPTO:BINANCE.UM:ETHUSDT, CRYPTO:BINANCE.UM:BTCUSDT}


In [18]:
isd = IterableSimulatorData(ld, open_close_time_indent_secs=300)
isd.set_warmup_period(Subtype.OHLC['1d'], "24h")

In [19]:
# isd.add_instruments_for_subscription(Subtype.OHLC['1h'], [s1, s2])
# isd.add_instruments_for_subscription(Subtype.OHLC['1h'], s3)
# isd.add_instruments_for_subscription(Subtype.OHLC['4h'], s3)
isd.add_instruments_for_subscription(Subtype.OHLC['1d'], s3)
# isd.add_instruments_for_subscription(Subtype.OHLC_TICKS['4h'], s1)

In [20]:
isd.get_instruments_for_subscription(Subtype.OHLC_TICKS['4h'])

[]

In [15]:
isd._subtyped_fetchers

{'ohlc.1D': ohlc({'timeframe': '1D'}) (---) [BINANCE.UM:LTCUSDT] :-> RestoredBarsFromOHLC}

In [ ]:
# isd.remove_instruments_from_subscription(Subtype.OHLC_TICKS['4h'], s1)

In [ ]:
# isd.remove_instruments_from_subscription(Subtype.ALL, s1)

In [21]:
QubxLogConfig.set_log_level('DEBUG')
qiter = isd.create_iterable("2023-07-01", "2023-07-02")

In [22]:
for d in qiter:
    print(pd.Timestamp(d[2].time, 'ns'), d)

2024-12-03 17:41:44.980 [ 🐞 ] (simulated_data) Preloading initial data for ohlc.1D 2023-07-01 00:00:00 : 2023-07-02 00:00:00 ...
2024-12-03 17:41:44.980 [ 🐞 ] (helpers) (InMemoryCachedReader) Updating 3 symbols pre interval 2023-06-30 00:00:00 : 2023-07-01 00:00:00


2023-06-30 00:05:00.100000 (CRYPTO:BINANCE.UM:LTCUSDT, 'hist_ohlc', {o:84.680000 | h:84.680000 | l:84.680000 | c:84.680000 | v:0.000000})
2023-06-30 11:00:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'hist_ohlc', {o:84.680000 | h:84.680000 | l:84.230000 | c:84.230000 | v:517176.566543})
2023-06-30 13:00:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'hist_ohlc', {o:84.680000 | h:111.830000 | l:84.230000 | c:111.830000 | v:28076941.157016})
2023-06-30 23:54:59.900000 (CRYPTO:BINANCE.UM:LTCUSDT, 'hist_ohlc', {o:84.680000 | h:111.830000 | l:84.230000 | c:108.660000 | v:31720162.748000})
2023-07-01 00:05:00.100000 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:108.650000 | l:108.650000 | c:108.650000 | v:0.000000})
2023-07-01 11:00:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:113.200000 | l:108.650000 | c:113.200000 | v:6769664.483057})
2023-07-01 13:00:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:113.200000 | l:102.600000 | c:102.600000 | v:9090692.305819})
2023-07-01 23:54:59.900000 (CR